In [1]:
from common.functions import *
import numpy as np
from pandarallel import pandarallel
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.model_selection import cross_val_score, GridSearchCV, learning_curve, train_test_split, validation_curve
from sklearn.svm import SVR
from sqlalchemy import create_engine

In [2]:
def estimations(samples_test,
                model):

    samples_test['correction'] = samples_test.parallel_apply(get_correction_estimate,
                                                             model=model,
                                                             axis=1)
    samples_test['correction_error'] = \
        -samples_test['error'] - samples_test['correction'].dt.total_seconds()

    return samples_test

In [3]:
def get_correction_estimate(row, model):
    cod_stop = row['cod_stop']
    cod_line = row['cod_line']
    cod_issue = row['cod_issue']
    eta = row['eta']
    actual_date = row['actual_date']
    remaining_seconds_est = row['remaining_seconds_est']
    static = row['static']

    X_pred = np.concatenate([[remaining_seconds_est.total_seconds()],
                             [get_day_time(eta)],
                             [eta.weekday()],
                             [get_day_type_bool(eta)],
                             [static]]).reshape(1, -1)

    estimation = model.predict(X_pred)

    return pd.Timedelta(estimation[0], unit='s')

In [4]:
engine = create_engine(engine_string)

In [5]:
cod_stop = '8_06277'
cod_line = '8__658___'

arrival_times = pd.read_sql_query(
    "SELECT * FROM arrival_times WHERE cod_stop = '{cod_stop}' AND cod_line = '{cod_line}'".format(cod_stop=cod_stop, cod_line=cod_line), con=engine)

In [6]:
crtm_poll = pd.read_sql_query("SELECT * FROM crtm_poll "
                              "WHERE cod_stop = '{cod_stop}' "
                              "AND cod_line = '{cod_line}'".format(cod_stop=cod_stop,
                                                                   cod_line=cod_line),
                              con=engine)
crtm_poll['remaining_seconds_est'] = crtm_poll['eta'] - \
    crtm_poll['actual_date']
crtm_poll['eta_date'] = crtm_poll['eta'].dt.day

In [7]:
# Initialization
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [8]:
crtm_poll_grouped = crtm_poll.groupby(
    ['cod_issue', 'cod_stop', 'cod_line', 'eta_date'])
crtm_poll_filtered = crtm_poll_grouped.parallel_apply(
    lambda x: add_static_column(x)).reset_index(drop=True)

In [9]:
def get_arrival_time(row):
    cod_line = row['cod_line']
    cod_issue = row['cod_issue']
    eta_date = row['eta_date']

    selected_arrival_times = arrival_times[(arrival_times['cod_line'] == cod_line) &
                                           (arrival_times['cod_issue'] == cod_issue) &
                                           (arrival_times['eta_date']
                                            == eta_date)
                                           ]['arrival_time']

    if (len(selected_arrival_times.index) == 1):
        arrival_time = selected_arrival_times.iloc[0]
    else:
        arrival_time = None

    return arrival_time

In [10]:
crtm_poll_filtered['arrival_time'] = crtm_poll_filtered.parallel_apply(
    get_arrival_time, axis=1)

In [11]:
crtm_poll_filtered.dropna(inplace=True)

In [12]:
crtm_poll_filtered['remaining_seconds'] = (
    crtm_poll_filtered['arrival_time'] - crtm_poll_filtered['actual_date']).astype('timedelta64[s]')
crtm_poll_filtered['error'] = (crtm_poll_filtered['arrival_time'] -
                               crtm_poll_filtered['eta']).astype('timedelta64[s]')

In [13]:
#crtm_poll_filtered = crtm_poll_filtered[crtm_poll_filtered['error'] < 1500] # error<25min
crtm_poll_filtered = filter_outliers_df(crtm_poll_filtered, 'error')
crtm_poll_filtered = crtm_poll_filtered[crtm_poll_filtered['remaining_seconds'] >= 0]
crtm_poll_filtered = crtm_poll_filtered[crtm_poll_filtered['remaining_seconds'] <= 90*60]

In [14]:
samples_train, samples_test = train_test_split(
    crtm_poll_filtered, test_size=0.25, random_state=42)

In [15]:
y_train = -samples_train['error']

X_train = pd.concat([samples_train['remaining_seconds_est'].dt.total_seconds(),
                     samples_train['eta'].apply(get_day_time),
                     samples_train['eta'].dt.weekday,
                     samples_train['eta'].apply(get_day_type_bool),
                     samples_train['static']
                     ], axis=1)

## won't even fit

In [18]:
param_grid = {'C': [1],
              'epsilon': [0.1],
              'kernel': ['linear'],
              'gamma': ['auto']
             }

grid_dtree = GridSearchCV(SVR(),
                          param_grid,
                          cv=5,
                          refit=True,
                          verbose=1,
                          scoring='neg_mean_absolute_error',
                          n_jobs=-1)
grid_dtree.fit(X_train, y_train)

sc_dtree = get_best_score(grid_dtree)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.


KeyboardInterrupt: 

In [40]:
fig = px.line(grid_dtree.cv_results_,
                 x='param_max_depth',
                 y=-grid_dtree.cv_results_['mean_test_score'],
                 color='param_n_estimators'
                )

# Set y-axes titles
fig.update_yaxes(title_text="<b>MAE</b> (seconds)", secondary_y=False)

# Edit the layout
fig.update_layout(title='Cross-validation MAE score depending on the decission tree maximumn depth',
                  xaxis_title='Decission tree maximun depth')

fig.show()

In [41]:
model = RandomForestRegressor(n_estimators=grid_dtree.best_params_['n_estimators'],
                              max_depth=grid_dtree.best_params_['max_depth'],
                              min_samples_leaf=grid_dtree.best_params_['min_samples_leaf'],
                              max_leaf_nodes=grid_dtree.best_params_['max_leaf_nodes'],
                              criterion=grid_dtree.best_params_['criterion'])

model.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=26, max_features='auto', max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=10,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=6, n_jobs=None, oob_score=False,
                      random_state=None, verbose=0, warm_start=False)

In [42]:
train_sizes, train_scores, valid_scores = learning_curve(RandomForestRegressor(n_estimators=grid_dtree.best_params_['n_estimators'],
                                                                               max_depth=grid_dtree.best_params_[
                                                                                   'max_depth'],
                                                                               min_samples_leaf=grid_dtree.best_params_[
                                                                                   'min_samples_leaf'],
                                                                               max_leaf_nodes=grid_dtree.best_params_[
                                                                                   'max_leaf_nodes'],
                                                                               criterion=grid_dtree.best_params_['criterion']),
                                                         X_train,
                                                         y_train,
                                                         train_sizes=np.linspace(
    0.1, 1, 10),
    cv=5,
    scoring='neg_mean_absolute_error',
    n_jobs=-1
)

In [43]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=train_sizes,
        y=-np.mean(train_scores, axis=1),
        name='Trainning error'
    )
)
fig.add_trace(
    go.Scatter(
        x=train_sizes,
        y=-np.mean(valid_scores, axis=1),
        name='Validation error'
    )
)
fig.show()

In [44]:
# Initialization
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 12 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [45]:
plot_df = estimations(samples_test, model)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [46]:
plot_df['remaining_seconds_corrected'] = plot_df['remaining_seconds_est'] - plot_df['correction']
plot_df['eta_corrected'] = plot_df['actual_date'] + plot_df['remaining_seconds_corrected']
plot_df['error_corrected'] = (plot_df['arrival_time'] -
                               plot_df['eta_corrected']).astype('timedelta64[s]')

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/st

In [47]:
plot_df_grouped = plot_df.groupby(np.floor(plot_df['remaining_seconds']/60))

In [48]:
mae_reds_crtm = plot_df_grouped.apply(lambda x: np.mean(np.abs(x.query("static == 1")['error']/60))).reset_index(name='mae')
mae_greens_crtm = plot_df_grouped.apply(lambda x: np.mean(np.abs(x.query("static == 0")['error']/60))).reset_index(name='mae')
mae_global_crtm = plot_df_grouped.apply(lambda x: np.mean(np.abs(x['error']/60))).reset_index(name='mae')
mape_crtm = plot_df_grouped.apply(lambda x: np.mean(np.abs(x['error']/60)/np.abs(x.name))*100).reset_index(name='mape')

In [49]:
mae_reds_model = plot_df_grouped.apply(lambda x: np.mean(np.abs(x.query("static == 1")['error_corrected']/60))).reset_index(name='mae')
mae_greens_model = plot_df_grouped.apply(lambda x: np.mean(np.abs(x.query("static == 0")['error_corrected']/60))).reset_index(name='mae')
mae_global_model = plot_df_grouped.apply(lambda x: np.mean(np.abs(x['error_corrected']/60))).reset_index(name='mae')
mape_model = plot_df_grouped.apply(lambda x: np.mean(np.abs(x['error_corrected']/60)/np.abs(x.name))*100).reset_index(name='mape')

In [50]:
fig = go.Figure()

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# crtm


fig.add_trace(
    go.Scatter(x=mae_reds_crtm['remaining_seconds'], y=mae_reds_crtm['mae'], name='MAE static (CRTM)'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=mae_greens_crtm['remaining_seconds'], y=mae_greens_crtm['mae'], name='MAE rt (CRTM)'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=mae_global_crtm['remaining_seconds'], y=mae_global_crtm['mae'], name='MAE global (CRTM)'),
    secondary_y=False)


# model

fig.add_trace(
    go.Scatter(x=mae_reds_model['remaining_seconds'], y=mae_reds_model['mae'], name='MAE static (model)'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=mae_greens_model['remaining_seconds'], y=mae_greens_model['mae'], name='MAE rt (model)'),
    secondary_y=False)

fig.add_trace(
    go.Scatter(x=mae_global_model['remaining_seconds'], y=mae_global_model['mae'], name='MAE global (model)'),
    secondary_y=False)


fig.update_xaxes(autorange="reversed")

# Set y-axes titles
fig.update_yaxes(title_text="<b>MAE</b> (minutes)", secondary_y=False)

# Edit the layout
fig.update_layout(title='Estimation error per minute',
                  xaxis_title='Remaining time (minutes)')

fig.show()